In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('sales_final.xlsx')

In [3]:
df.head(1)

,Unnamed: 0,date,receipt,product,group_of_products,store,city,sold_quantity,tunover_no_VAT,tunover_VAT,purchase_price_VAT,purchase_cost_VAT,margin_VAT,profit,temperature
0,1,2017-09-21,5,"ИВ Лимонад Лаймони 0,6 л стекло","ВКУСОВАЯ, СЛАДКАЯ ВОДА, НАПИТКИ",АЗС № 1,Симферополь г,1,NaN,70.0,48.3,48.3,21.7,21.7,NaN


## дублирую таблицу, удаляя ненужные столбцы и переименовывая их 

In [64]:
df1 = df.drop(['Unnamed: 0', 'date', 'receipt', 'group_of_products', 'store', 'city', 
               'sold_quantity', 'tunover_no_VAT', 'tunover_VAT', 'purchase_price_VAT', 'purchase_cost_VAT', 
               'margin_VAT', 'temperature'], axis=1)

In [65]:
df1 = df.groupby('product').agg({'product':'count','profit':'sum'})\
.rename(columns={'product':'Количество', 'profit': 'Доход'}).reset_index()
df1 = df1.rename(columns={'product':'Товар', 'profit': 'Доход'})

### создаем две таблицы, сортируем одну по доходу, вторую по количеству 

### abc - Анализ по доходу, xyz – анализ по спросу 

In [66]:
df_abc = df1.sort_values(by='Доход', ascending=False)
df_abc = df_abc.drop('Количество', axis=1)

In [67]:
df_abc.head(3)

,Товар,Доход
2579,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,1.227513e+06
2590,Кофе КАПУЧИНО 200 мл АЗС 2 АЗС 4,8.269544e+05
2587,Кофе ДВОЙНОЙ ЭСПРЕССО 120 мл АЗС 2 АЗС 4,5.873150e+05


In [68]:
df_xyz = df1.sort_values(by='Количество', ascending=False)
df_xyz = df_xyz.drop('Доход', axis=1)

In [69]:
df_xyz.head(3)

,Товар,Количество
2579,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,25148
2624,Кофе с молоком 160 мл АЗС № 11,15478
2590,Кофе КАПУЧИНО 200 мл АЗС 2 АЗС 4,15288


## считаем, доля вклада каждого товара в общую выручку 

In [70]:
df_abc['Доля'] = df_abc['Доход'] / df_abc['Доход'].sum()
df_xyz['Доля'] = df_xyz['Количество'] / df_xyz['Количество'].sum()

In [71]:
df_abc['Аккум.доля'] = df_abc['Доля'].cumsum()
df_xyz['Аккум.доля'] = df_xyz['Доля'].cumsum()

In [72]:
df_abc["Категория"] = ''
df_xyz["Категория"] = ''

In [73]:
df_abc.loc[df_abc['Аккум.доля'] < 0.8,'Категория' ] = 'A'
df_abc.loc[(df_abc['Аккум.доля'] > 0.8) & (df_abc['Аккум.доля'] < 0.95),'Категория' ] = 'B'
df_abc.loc[df_abc['Аккум.доля'] > 0.95,'Категория' ] = 'C'

df_xyz.loc[df_xyz['Аккум.доля'] < 0.8,'Категория' ] = 'A'
df_xyz.loc[(df_xyz['Аккум.доля'] > 0.8) & (df_xyz['Аккум.доля'] < 0.95),'Категория' ] = 'B'
df_xyz.loc[df_xyz['Аккум.доля'] > 0.95,'Категория' ] = 'C'

In [74]:
df_abc.head()

,Товар,Доход,Доля,Аккум.доля,Категория
2579,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,1.227513e+06,0.066017,0.066017,A
2590,Кофе КАПУЧИНО 200 мл АЗС 2 АЗС 4,8.269544e+05,0.044474,0.110491,A
2587,Кофе ДВОЙНОЙ ЭСПРЕССО 120 мл АЗС 2 АЗС 4,5.873150e+05,0.031586,0.142078,A
2624,Кофе с молоком 160 мл АЗС № 11,5.200956e+05,0.027971,0.170049,A
2599,Кофе ЛАТТЕ 300 мл АЗС 2 АЗС 4,4.044433e+05,0.021751,0.191800,A


In [75]:
df_xyz.head()

,Товар,Количество,Доля,Аккум.доля,Категория
2579,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,25148,0.035245,0.035245,A
2624,Кофе с молоком 160 мл АЗС № 11,15478,0.021693,0.056938,A
2590,Кофе КАПУЧИНО 200 мл АЗС 2 АЗС 4,15288,0.021426,0.078364,A
5392,Чай Мустанг,12686,0.017780,0.096143,A
2582,Кофе Американо 160 мл АЗС 11,11807,0.016548,0.112691,A
